## MNIST로 딥러닝 스타트

* MNIST는 다음의 주소에서 다운로드 가능
  * http://yann.lecun.com/exdb/mnist
* keras의 이용도 가능

<img src="img/MNIST01.png">

In [4]:
import tensorflow as tf
#from keras.datasets import mnist
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [11]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [5]:
type(x_train)

numpy.ndarray

## 01-02 신경망 모델 구성하기 
  * MNIST의 손글씨는 28 X 28로 구성되어 있다.
  * 784개의 특징으로 구성되어 있음.
  * 레이블은 0~9까지의 10개의 분류

In [6]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

### 미니배치의 이해
 * 이미지를 하나씩 학습시키는 것보다 여러 개를 한꺼번에 학습시키는 쪽이 효과가 좋다.
 * 많은 메모리와 높은 컴퓨터 성능이 필요하므로 일반적으로 데이터를 적당한 크기로 잘라서 학습시킨다.
  * **미니배치**라고 한다.
 * tf.float32, [None, 784] => None의 자리에는 한번에 학습시킬 이미지의 개수를 지정하는 값이 들어감., 즉 배치 크기를 지정하는 자리이다.

### 신경망의 구성
```
 * 784(입력, 특징 개수) -> 
     256(첫번째 은닉층 뉴런) -> 256(두번째 은닉층 뉴런)
                        -> 10(결과값 0~9 분류 개수 )
```

In [36]:
W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L2, W3)

print(W3)
print(model)

<tf.Variable 'Variable_5:0' shape=(256, 10) dtype=float32_ref>
Tensor("MatMul_5:0", shape=(?, 10), dtype=float32)


##
 * tf.random_normal([784, 256], stddev=0.01)  => 표준편차가 0.01인 정규 분포를 가지는 임의의 값으로 뉴런을 초기화
 * tf.matmul() 각 계층으로 들어오는 입력값에 가중치를 곱하고, tf.nn.relu함수를 이용하여 활성화 함수를 ReLU를 사용.
 * 마지막 계층인 model 텐서에 W3를 곱함으로 요소 10개짜리 배열이 출력된다.
 * 10개의 요소는 0부터 9까지의 숫자를 나타낸다.
 * 가장 큰 값을 가진 인덱스(결과 항목)이 예측 결과에 가까운 숫자이다.
 * 출력층은 보통 활성화 함수를 사용하지 않는다.

In [14]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

### (QA) 어떤 버전으로 변경될까?

### 신경망 모델 세션 시작

In [15]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

### 배치 사이즈 지정
 * 미니배치의 크기 100개 설정.
 * 학습 데이터의 총 개수 train.num_examples 를 배치 크기로 나눈다.
 * 총 배치 크기가 몇개인지 확인)

In [17]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)
total_batch

550

* mnist.train.next_batch(batch_size)를 이용하여 학습할 데이터를 배치 크기만큼 가져온다
* **입력값**인 이미지 데이터는 **batch_xs**에 **출력값**인 레이블 데이터는 **batch_ys**에 저장
<br><br>
*  _ , cost_val = sess.run([optimizer, cost],feed_dict =  {X: batch_xs, Y: batch_ys})
* sess.run을 이용하여 최적화를 시키고 cost(손실)을 가져와서 저장한다.
<br><br>
* X: batch_xs, Y: batch_ys
* 입력값 X와 예측을 평가할 실제 레이블 값 Y에 사용할 데이터를 넣어준다.

In [32]:
# MNIST 데이터 전체를 학습하는 일을 15번 반복함.
# 학습 데이터 전체를 한 바퀴를 도는 일을 에포크(epoch)라 한다.
for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        # 배치 사이즈만큼 데이터 가져오기 
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # 입력값 : batch_xs, 출력값 : batch_ys
        # 최적화를 수행 후, 손실을 구한다.
        _ , cost_val = sess.run([optimizer, cost],
                                feed_dict =  {X: batch_xs, Y: batch_ys})

        # 총 손실 계산
        total_cost = total_cost + cost_val

    print("Epoch : ", "%4d" %(epoch + 1), 
                '평균 Cost = ', "{:.3f}".format(total_cost/total_batch))
    
print("최적화 완료!")

Epoch :     1 평균 Cost =  0.005
Epoch :     2 평균 Cost =  0.004
Epoch :     3 평균 Cost =  0.007
Epoch :     4 평균 Cost =  0.005
Epoch :     5 평균 Cost =  0.002
Epoch :     6 평균 Cost =  0.005
Epoch :     7 평균 Cost =  0.006
Epoch :     8 평균 Cost =  0.004
Epoch :     9 평균 Cost =  0.006
Epoch :    10 평균 Cost =  0.003
Epoch :    11 평균 Cost =  0.003
Epoch :    12 평균 Cost =  0.008
Epoch :    13 평균 Cost =  0.003
Epoch :    14 평균 Cost =  0.000
Epoch :    15 평균 Cost =  0.000
최적화 완료!


### 정확도 확인

In [33]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
is_correct

<tf.Tensor 'Equal_2:0' shape=(?,) dtype=bool>

In [34]:
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

## (QA) X, Y의 값은 어떻게 어디로 들어가는 것일까?

In [35]:

print('정확도 :', sess.run(accuracy, 
                       feed_dict = {X:mnist.test.images,
                                    Y:mnist.test.labels}))

정확도 : 0.9817


## 1-3 DropOut(드롭아웃)
 * 전체 신경망 중에 일부만을 사용하도록 하는 것. (어떤 원리로?)
 * 즉, 학습 단계마다 일부 뉴런을 제거(사용하지 않도록)함으로, 일부 특징이 특정 뉴런들에 고정되는 것을 막아 가중치의 균형을 잡도록 한다.(**과적합 방지**)

 * 학습시 일부 뉴런을 학습시키지 않기 때문에 신경망이 충분히 학습되기까지 시간이 오래 걸린다.